This notebook shows an example of the model presented in the following [Medium article](https://medium.com/@octosport/football-prediction-performance-how-to-calculate-hit-ratio-and-log-loss-1e5e22310497).



In [1]:
 !pip install git+https://github.com/octosport/octopy.git

  Cloning https://github.com/octosport/octopy.git to /tmp/pip-req-build-7ziz3mph
  Running command git clone -q https://github.com/octosport/octopy.git /tmp/pip-req-build-7ziz3mph
     |████████████████████████████████| 38.2 MB 1.2 MB/s 
  Created wheel for octopy: filename=octopy-1.0.0-py3-none-any.whl size=11771 sha256=a9ace9f464e3e264f517466726447d92e5a59406fe9e41f992fa7ad0b1d0e9ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-p7t30hqj/wheels/fe/92/b1/51d04bb184b9442db5fee66ec3acfdc85798cc4423d9fe40a0
Successfully built octopy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from octopy.metrics import compute_1x2_log_loss, compute_1x2_hit_ratio
 
data = pd.read_csv('https://raw.githubusercontent.com/octosport/octopy/master/data/epl.csv') #get some data

#compute the winner
data['1'] = (data['home_goals']>data['away_goals'])*1
data['X'] = (data['home_goals']==data['away_goals'])*1
data['2'] = (data['home_goals']<data['away_goals'])*1
 

In [3]:
 #This is a random example, you should use your probabilities here.
probabilities = pd.DataFrame(np.random.randint(1,100,size=data[['1','X','2']].shape),columns=['1','X','2'])
probabilities = (probabilities.T/probabilities.sum(1)).T
true_result = data[['1','X','2']].idxmax(1)

#no need to add the index but index have to be the same.
probabilities.index = [x+' - '+y for x,y in (data[['home','away']].values)]
true_result.index = [x+' - '+y for x,y in (data[['home','away']].values)]


In [4]:
#the probabilities dataframe
probabilities.tail().round(2)

,1,X,2
Brighton and Hove Albion - Watford,0.28,0.23,0.49
Southampton - Manchester United,0.30,0.45,0.26
Wolverhampton - Tottenham Hotspur,0.42,0.57,0.01
Arsenal - Chelsea,0.26,0.26,0.49
West Ham United - Leicester City,0.51,0.40,0.09


In [5]:
#the true result series
true_result.tail()

Brighton and Hove Albion - Watford    1
Southampton - Manchester United       X
Wolverhampton - Tottenham Hotspur     2
Arsenal - Chelsea                     2
West Ham United - Leicester City      1
dtype: object

In [6]:
#compute the losses!
log_loss = compute_1x2_log_loss(probabilities.tail(50),true_result.tail(50))
hit_ratio = compute_1x2_hit_ratio(probabilities.tail(50),true_result.tail(50))
number_of_matches = probabilities.shape[0]

In [9]:
print(f'The log-loss for the {number_of_matches} matches is {np.round(log_loss,3)} and the hit-ratio is {hit_ratio*100}%')

The log-loss for the 1920 matches is -1.246 and the hit-ratio is 46.0%
